In [1]:
import pandas as pd
import numpy as np

In [2]:
file = "Resources/purchase_data.csv"
raw = pd.read_csv(file)

In [3]:
raw.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [4]:
player_list = raw["SN"].unique()
player_count = len(player_list)
player_count_df = pd.DataFrame({"Total Players":[player_count]})
player_count_df.head()

,Total Players
0,576


In [5]:
item_list = raw["Item ID"].unique()
num_purchases = len(raw["Purchase ID"])
total_revenue = raw["Price"].sum()
avg_price = round(total_revenue/num_purchases,2)
purchase_summary = pd.DataFrame({
    "Number of Unique Items":len(item_list),
    "Average Price":["$"+str(avg_price)],
    "Number of Purchases":[num_purchases],
    "Total Revenue":["$"+str(total_revenue)]
})
purchase_summary

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [6]:
duplicate_sn_removed = raw.drop_duplicates("SN")
gender_count=duplicate_sn_removed["Gender"].value_counts()
gender_percent = round((gender_count/player_count)*100,2)
gender_count_df = pd.DataFrame({
    "Total Count":gender_count,
    "Percentage of Players":gender_percent.apply("{0:.2f}%".format)
})
gender_count_df.head()

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [7]:
gender_groupby=raw.groupby(["Gender"])
purchase_count = gender_groupby["Purchase ID"].count()
total_purchase_value=gender_groupby["Price"].sum()
avg_purchase_price = round((total_purchase_value/purchase_count),2)
avg_per_person = round(total_purchase_value/gender_count,2)
gender_breakdown = pd.DataFrame({
    "Purchase Count":purchase_count,
    "Average Purchase Price":avg_purchase_price.apply("${0:.2f}".format),
    "Total Purchase Value":total_purchase_value.apply("${0:.2f}".format),
    "Avg Total Purchase per Person":avg_per_person.apply("${0:.2f}".format)
})
gender_breakdown.head()


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [8]:
bins=[0,9,14,19,24,29,34,39,100]
age_sort=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

new_row = duplicate_sn_removed["Age"]
test_df = pd.DataFrame({"Age":new_row})

test_df["Age Range"]= pd.cut(test_df["Age"], bins, labels=age_sort)

age_range_count=test_df.groupby("Age Range").count()
age_count = age_range_count["Age"]

percent_age = round((age_count/player_count)*100,2)
age_breakdown = pd.DataFrame({
    "Total Count":age_count,
    "Percentage of Players":percent_age.apply("{0:.2f}%".format)
})
age_breakdown


,Total Count,Percentage of Players
Age Range,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [9]:
raw["Age Range"]= pd.cut(raw["Age"], bins, labels=age_sort)
age_price = raw[["Age Range", "Price"]]
price_groupby = age_price.groupby("Age Range")
age_total_purchase = price_groupby.sum()
age_purchase_count = price_groupby.count()
age_avg_purchase_price = round((age_total_purchase/age_purchase_count),2)

duplicate_sn_removed["Age Range"]= pd.cut(duplicate_sn_removed["Age"], bins, labels=age_sort)
unique_sn_groupby = duplicate_sn_removed.groupby("Age Range")
unique_groupby_reduced = unique_sn_groupby[["Price"]]
age_purchaser = unique_groupby_reduced.count()

age_avg_per_person = round(age_total_purchase/age_purchaser,2)

age_purchase_breakdown = pd.DataFrame({
    "Purchase Count":age_purchase_count["Price"],
    "Average Purchase Price":age_avg_purchase_price["Price"].apply("${0:.2f}".format),
    "Total Purchase Value":age_total_purchase["Price"].apply("${0:.2f}".format),
    "Avg Total Purchase per Person":age_avg_per_person["Price"].apply("${0:.2f}".format)
})

age_purchase_breakdown

C:\Users\prins\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Range,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [10]:
spenders= raw.groupby("SN")
count_purchases = spenders["Purchase ID"].count()
total_spent = spenders["Price"].sum()
avg_spent = round(total_spent/count_purchases,2)
spenders_breakdown = pd.DataFrame({
    "Purchase Count":count_purchases,
    "Average Purchase Price":avg_spent.apply("${0:.2f}".format),
    "Total Purchase Value":total_spent.apply("${0:.2f}".format)
})
spenders_breakdown.sort_values(by="Purchase Count", ascending=False).head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Iral74,4,$3.40,$13.62
Idastidru52,4,$3.86,$15.45
Asur53,3,$2.48,$7.44
Inguron55,3,$3.70,$11.11


In [11]:
item_df= raw[["Item ID","Item Name", "Price"]]
item_groupby = item_df.groupby(["Item ID","Item Name"])

item_total = round(item_groupby["Price"].sum(),2)
item_buy_count = item_groupby["Price"].count()
item_price = round(item_total/item_buy_count,2)

item_breakdown = pd.DataFrame({
    "Purchase Count":item_buy_count,
    "Item Price":item_price.apply("${0:.2f}".format),
    "Total Purchase Value":item_total
})

final_item_breakdown = item_breakdown.sort_values(by="Total Purchase Value",ascending=False).head()
final_item_breakdown["Total Purchase Value"] = final_item_breakdown["Total Purchase Value"].apply("${0:.2f}".format)
final_item_breakdown

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


In [ ]:
# The three observable trends I noticed in this dataset are:
#     1. The population of players is largely comprised of male players.
#     2. Despite being a smaller population, both females and those with gender undisclosed/other spend more money on average per person.
#     3. The majority of players are between the ages of 15 and 24.  This population also accounts for the majority of purchases
#     Bonus: spend more resources marketing to 9 year olds and 35-39 years olds, as they spend more money per person on average.